In [ ]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.4 MB 11.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 1.2 MB 44.6 MB/s 
     |████████████████████████████████| 61 kB 389 kB/s 
     |████████████████████████████████| 596 kB 26.8 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=aba70021b36ef27139e8daa755d2acd5d0b270ab5ecc09b4d62a97b74834f114
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# cd '/content/drive/MyDrive/685/catr'
#For Himanshu's access
import os
path = "/content/drive/MyDrive/CollabProjects/685/catr"
os.chdir(path)

In [ ]:
!pip install -r requirements.txt

In [ ]:
# from transformers import ViTModel, ViTConfig, ViTFeatureExtractor,BertTokenizer,BertForMaskedLM
from transformers import DeiTFeatureExtractor, DeiTModel #AutoFeatureExtractor, DeiTForImageClassificationWithTeacher, 
from sentence_transformers import SentenceTransformer
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import auc, precision_score, recall_score,roc_auc_score
import xgboost as xgb
from PIL import Image
import argparse
import glob
import json
import pandas as pd
import numpy as np
from models import caption
from datasets import coco, utils
from tqdm import tqdm
from configuration import Config
from xgboost import XGBClassifier
import os

In [ ]:
def extract_vision_transformer_feats(image_path,nsel_st=None,nsel_end=None):
  """
  Function to extract the features from vision transformers
  """
  #code to get all images present in a certain csv file
  csv_data=pd.read_csv("/content/drive/MyDrive/685/emogen/Classifier/train/caption/negative_captions.csv")
  img_list = [os.path.join(image_path,str(i+'.jpg')) for i in list(csv_data['image_hash'])]

  if not nsel_st:
    img_files_list = img_list#glob.glob(image_path+"*")
  else:
    img_files_list = img_list[nsel_st:nsel_end]#glob.glob(image_path+"*")[nsel_st:nsel_end]
  
  # Create image batch array for Vision Transformer
  img_batch = []
  for file in tqdm(img_files_list):
    img = np.asarray(Image.open(file))
    newsize = (240, 240, 3)
    img = np.resize(img,newsize)
    img_batch.append(img)
  print("Creation of image batches to be used for Vision Transformer complete")

  
  # Extract features from ViTModel
  # feature_extractor   = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
  # model               = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
  feature_extractor   =  DeiTFeatureExtractor.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  model               = DeiTModel.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  inputs              = feature_extractor(images=img_batch, return_tensors="pt")
  outputs             = model(**inputs)

  # Get image representations and their corresponding hashes i.e. get [CLS] token representation for each image
  img_representations = outputs.last_hidden_state[:,0,:]
  img_hash_li         = []
  for file in img_files_list:
    img_hash_li.append(file.split("/")[-1][:-4])
  
  # Create column names for image dimensions 
  col_img  = ["imdim_"+str(i) for i in list(range(192))]

  # Create a dataframe of image features
  img_data = pd.DataFrame(img_representations.detach().numpy(),columns = col_img)
  img_data['img_hash'] = img_hash_li

  return img_data

def extract_sentence_transformer_feats(reference_file_pth = '/content/drive/MyDrive/685/emogen/Classifier/train/caption/',csv_file = 'negative_captions.csv',lab_assign=0):
  """
  Extract features for emotion related texts
  """
  # Get reference text data
  combined_path = reference_file_pth + csv_file
  #file = open(combined_path)
  #data_json = json.load(file)
  data = pd.read_csv(combined_path)#pd.DataFrame.from_dict(data_json['annotations']).reset_index(drop=True)
  data = data[1:2001].reset_index(drop=True)
  try:
    del data['Unnamed: 0']
  except:
    pass
  
  # Use Sentence Transformer to extract features
  model = SentenceTransformer('all-mpnet-base-v2')
  sentence_embeddings = model.encode(data['comment'])
  col_text = ["tdim_"+str(i) for i in list(range(768))]

  # Create text feature dataframe
  text_data = pd.DataFrame(sentence_embeddings,columns=col_text)
  text_data['img_hash'] = data['image_hash']
  text_data['label'] = lab_assign

  return text_data

def xgb_train_kfold(X_trn, y_trn,n_splits=5,max_depth=6,n_estimator=500, rand_st=3815):
  """
  Perform training with XGBoost and evaluate in K-Fold cross-validation settings 
  """
  errors    = []
  precision = []
  recall    = []
  auc       = []
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=3815)

  for train_index, test_index in tqdm(kf.split(X_trn)):
     X_train_n, X_test_n = X_trn.values[train_index], X_trn.values[test_index]
     y_train_n, y_test_n = y_trn.values[train_index], y_trn.values[test_index]

     model = XGBClassifier(
         max_depth=max_depth, n_estimators=n_estimator, random_state = rand_st
     )
     model.fit(X_train_n, y_train_n)
     y_pred = model.predict(X_test_n)
     accuracy = (sum(y_pred == y_test_n))/len(y_test_n)
     errors.append(1 - accuracy)
     precision.append(precision_score(y_test_n,y_pred))
     recall.append(recall_score(y_test_n,y_pred))
     auc.append(roc_auc_score(y_test_n,y_pred))

  return model,errors, precision, recall, auc

In [ ]:
# pos_img_feats=pd.read_csv('/content/drive/MyDrive/685/emogen/evaluations/Vision_trans/pos_image_feats.csv')
# pos_text_feats=pd.read_csv('/content/drive/MyDrive/685/emogen/evaluations/Vision_trans/pos_text_feats.csv')
# non_pos_img_feats=pd.read_csv('/content/drive/MyDrive/685/emogen/evaluations/Vision_trans/non_pos_image_feats.csv')
# non_pos_text_feats=pd.read_csv('/content/drive/MyDrive/685/emogen/evaluations/Vision_trans/non_pos_text_feats.csv')




'''import pandas as pd
pos_img_feats=pd.read_csv('../emogen/evaluations/Vision_trans/pos_image_feats.csv')
pos_text_feats=pd.read_csv('../emogen/evaluations/Vision_trans/pos_text_feats.csv')
non_pos_img_feats=pd.read_csv('../emogen/evaluations/Vision_trans/non_pos_image_feats.csv')
non_pos_text_feats=pd.read_csv('../emogen/evaluations/Vision_trans/non_pos_text_feats.csv')'''

In [ ]:
img_data=pd.concat([pos_img_feats,non_pos_img_feats],axis=0).sample(frac=1)
text_data=pd.concat([pos_text_feats,non_pos_text_feats],axis=0).sample(frac=1)
mmodal_data = img_data.merge(text_data,how='left',on='img_hash')

In [ ]:
# from sklearn.model_selection import train_test_split
# X_trn, X_tst, y_trn, y_tst = train_test_split(mmodal_data.drop(columns=['label','img_hash'],axis=1), mmodal_data['label'], test_size=0.15)
print(mmodal_data.shape, img_data.shape, text_data.shape)
print(img_data.columns)

(2000, 1539) (2000, 770) (3000, 770)
Index(['Unnamed: 0', 'imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4',
       'imdim_5', 'imdim_6', 'imdim_7', 'imdim_8',
       ...
       'imdim_759', 'imdim_760', 'imdim_761', 'imdim_762', 'imdim_763',
       'imdim_764', 'imdim_765', 'imdim_766', 'imdim_767', 'img_hash'],
      dtype='object', length=770)


In [ ]:
X_train = X_trn.values
y_train = y_trn.values
X_test  = X_tst.values
y_test  = y_tst.values 
print(X_test.shape)

(300, 1537)


In [ ]:
model,errors, precision, recall, auc = xgb_train_kfold(X_trn,y_trn)

5it [05:37, 67.59s/it]


In [ ]:

'''labels = [int(x) for x in mmodal_data['label']]
mmodal_data['Target']  = labels'''

print("The training errors on average is: ", np.round(np.mean(errors),4)*100)
print("The Precision on average is: ", np.round(np.mean(precision),4)*100)
print("The Recall on average is: ", np.round(np.mean(recall),4)*100)
print("The AUC Score on average is: ", np.round(np.mean(auc),4)*100)

The training errors on average is:  27.71
The Precision on average is:  71.34
The Recall on average is:  75.03
The AUC Score on average is:  72.26


In [ ]:
pred_test = model.predict(X_test)
pred_prob = model.predict_proba(X_test)
accuracy = (sum(pred_test == y_test))*100/len(y_test)
precision = precision_score(y_test,pred_test)
recall    = recall_score(y_test,pred_test)
auc       = roc_auc_score(y_test,pred_test)
accuracy,precision,recall, auc

(93.66666666666667, 0.9266666666666666, 0.9455782312925171, 0.9368414032279578)

In [ ]:
import pickle
file_name = "../emogen/evaluations/Vision_trans/xgb_cls_positive_vit_mmodal.pkl"

# save
# pickle.dump(model, open(file_name, "wb"))

# load
model = pickle.load(open(file_name, "rb"))